In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from state import get_state, queue, get_state_per_traffic_light
import os
import traci

In [36]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
#sumo_gui_bin = os.getenv("SUMO-GUI")
sumo_gui_bin = '/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui'
simulConfig = os.getenv("SIMUL-CONFIG2")


print(sumo_gui_bin)

/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui


In [37]:
import numpy as np

from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [38]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [64]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action_one_tf_light = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [39]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [64]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action_second_tf_light = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [40]:
def epsilon_greedy_policy(state_tf_one, state_tf_two,epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs), np.random.randint(n_outputs)  # random action
    else:
        Q_values_tf_one = model_action_one_tf_light.predict(state_tf_one[np.newaxis], verbose=0)[0]
        Q_values_tf_two = model_action_second_tf_light.predict(state_tf_two[np.newaxis], verbose=0)[0]

        print(f"state : {state_tf_one, state_tf_two}")
        print(f"Q_values : {Q_values_tf_one, Q_values_tf_two}")
        return Q_values_tf_one.argmax(), Q_values_tf_two.argmax()  # optimal action according to the DQN

In [41]:
def sample_experiences(batch_size, name_batch):
    indices = np.random.randint(len(name_batch), size=batch_size)
    batch = [name_batch[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [42]:
batch_size = 32
discount_factor = 0.5
optimizer_one = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step_tf_one(batch_size, model, name_batch):
    experiences = sample_experiences(batch_size, name_batch)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer_one.apply_gradients(zip(grads, model.trainable_variables))

In [43]:
batch_size = 32
discount_factor = 0.5
optimizer_two = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step_tf_two(batch_size, model, name_batch):
    experiences = sample_experiences(batch_size, name_batch)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer_two.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
replay_buffer_tf_one = deque(maxlen=10000)
replay_buffer_tf_two = deque(maxlen=10000)

epoch = 100






# PERIODE DE STABILITE

# traffic_scale = [0.1] * 100
# AUGMENTATION DE TRAFFIC
# traffic_scale.extend(np.linspace(0.1, 1, 400, dtype=float))
# OR
#traffic_scale = 0.2



for episode in range(epoch):
    #SEED & TRAFFIC SCALING
    seed_value = np.random.randint(1000)
    #seed_value = 41
    if traci.isLoaded():
        traci.close()


    sumoCmd = [sumo_bin, "-c", simulConfig, "--start", "--no-warnings", "--seed", str(seed_value), '--scale', str(np.random.uniform(0.1, 0.2))]

    traci.start(sumoCmd)





    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    traffic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))



    state_tf_one=np.array(get_state_per_traffic_light(traffic_light_ids[0]))
    state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))
    action_tf_one=-1
    action_tf_two=-1

    # print(state)
    for step in range(80000): ## TO CHANGED
        epsilon = max(1 - episode / epoch, 0.01)

        if step%2000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            # next_state = np.array(queue(lane_ids))
            next_state_tf_one = np.array(get_state_per_traffic_light(traffic_light_ids[0]))
            next_state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))

            # reward = calculate_reward(values, reward, total_reward)[0]
            # reward=(np.sum(state)-np.sum(next_state))
            reward_tf_one = np.sum(state_tf_one)- np.sum(next_state_tf_one)
            reward_tf_two = np.sum(state_tf_two)- np.sum(next_state_tf_two)

            replay_buffer_tf_one.append((state_tf_one, action_tf_one, reward_tf_one, next_state_tf_one))
            replay_buffer_tf_two.append((state_tf_two, action_tf_two, reward_tf_two, next_state_tf_two))
            #########################################################################
            state_tf_one = next_state_tf_one
            state_tf_two = next_state_tf_two

            action_tf_one, action_tf_two = epsilon_greedy_policy(state_tf_one,state_tf_two, epsilon)

            #print("action", action)
            traci.trafficlight.setPhase(traffic_light_ids[0],2*action_tf_one)
            traci.trafficlight.setPhase(traffic_light_ids[1],2*action_tf_two)

            if len(replay_buffer_tf_one) >= batch_size*10:
                training_step_tf_one(batch_size, model_action_one_tf_light, replay_buffer_tf_one)
                training_step_tf_two(batch_size, model_action_second_tf_light, replay_buffer_tf_two)

                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


In [58]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)

#SEED é SCALE
seed_value = np.random.randint(1000)
#seed_value = 41
traffic_scale = 0.5


#LAUNCH
sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start", "--seed", str(seed_value), '--scale', str(traffic_scale), '--gui-settings-file', 'full_state_sumo_gui_settings.xml']

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()


state_tf_one=np.array(get_state_per_traffic_light(traffic_light_ids[0]))
state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))
action_tf_one=-1
action_tf_two=-1# print(state)


for step in range(100000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)

    if step%2000 == 0:
        state_tf_one=np.array(get_state_per_traffic_light(traffic_light_ids[0]))
        state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))
        action_tf_one, action_tf_two = epsilon_greedy_policy(state_tf_one,state_tf_two, epsilon)
        #action = np.random.randint(3)
        print("action", action_tf_one, action_tf_two)
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu))

        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(traffic_light_ids[0],2*action_tf_one)
        traci.trafficlight.setPhase(traffic_light_ids[1],2*action_tf_two)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q_values : (array([-3.4133527, -3.1299524, -2.9892888, -3.1534247], dtype=float32), array([-3.1535773, -4.5133424, -3.3685424, -3.5586698], dtype=float32))
action 2 0
state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 